# Task 1: Small case scenario of dataset analysis

## 1.1 - Create a dataset (The Tate Collection)
As a first example, we will create our own small dataset. In particular, the dataset will consist of 10 artists which are present in the Tate Collection along with the corresponding year of birth, year of death, biological gender and number of artworks.

|          Name         | Gender |   Year_Birth  |   Year_Death  | N_works |
|-----------------------|--------|---------------|---------------|---------|
|     Beuys, Joseph     |  Male  |      1921	 |      1986     |   588   |
|    Constable, John    |  Male  |      1776	 |      1837	 |   249   |
|   Daniell, William    |  Male  |      1769	 |      1837	 |   612   |
|   Forbes, Elizabeth   | Female |      1859	 |      1912	 |   120   |
|     Flaxman, John     |  Male  |      1755	 |      1826	 |   287   |
|    Phillips, Thomas   |  Male  |      1770	 |      1845	 |   274   |
| Paolozzi, Sir Eduardo |  Male  |      1924	 |      2005	 |   385   |
|     Schendel, Mira    | Female |      1919	 |      1988	 |    3    |
|    Turner, William    |  Male  |      1775	 |      1851	 |  1861   |
|      Warhol, Andy     |  Male  |      1928	 |      1987	 |   272   |


**_[TO DO]_**: Let's create a DataFrame named **TateDataset** with the information showed in the above table.

**Note:** You can create arrays for each column and then build the DataFrame.
Remember to import the required libraries!

In [1]:
# @title Imports
import numpy as np
import pandas as pd


In [ ]:
### TODO
tate_dataset = ...

## 1.2 - Data Acquisition (local file)

Data acquisition is a process of loading and reading data from various sources. We will learn how to export and read data from a local file in different format using  **Pandas** package. 

A file format is a standardised way in which information is encoded to be stored in a file. Some examples of file formats are: CSV, XLSX, and PKL. 

Let's practice with some examples.

### .CSV - Comma Separated Values

CSV is one of the most popular spreadsheet file format. In this kind of file, data is stored in cells. Each cell is organized in rows and columns. A column in the spreadsheet file can have different types. 

Let us look at how to create a CSV file. We can export a DataFrame with the Pandas function `to_csv()`. The file will be saved in the same location of this notebook unless specified otherwise. We can name the file **TateData.csv**.

**Note:** Once you have run the following cell, check the output file .

In [ ]:
tate_dataset.to_csv('tate_dataset.csv', index=False, header=True)

To pull in the csv file, we will use the Pandas function `read_csv()`. We will import the previous file in a new DataFrame.

**Note:**  Depending on where you save your notebooks, you may need to modify the location below.

In [ ]:
tate_1 = pd.read_csv('tate_dataset.csv')
tate_1

When we create the .CSV file, the only parameter we have used is **index**. 

**[TO DO]** What happens if we set the *index* parameter to True? And if you set also the parameter **header** to False?
Try by yourself. 

In [ ]:
### TODO


Delete the csv file now that we are done using it.

### pickle — Python object serialization

This data format is Python-specific. This has the advantage that there are no restrictions imposed by external standards; however it means that non-Python programs may not be able to reconstruct pickled Python objects.



In [ ]:
import pickle

tate_dataset.to_pickle('tate_dataset.pkl')


In [ ]:
tate_2 = pd.read_pickle('tate_dataset.pkl')
tate_2

## 1.3 - Look at the data

Now we will simply have a look at the data and make sure it is clean. 

### Data type
Let's check the data type of the imported variables and the original DataFrame.

In [ ]:
# Check data type of the columns for Tate_1
tate_1.dtypes

In [ ]:
# Check data type of the columns for TateDataset
tate_dataset.dtypes

We can also display the values of a single column and check their data type as follow:

In [ ]:
tate_dataset['Gender']

# Task 2: Application Programming Interfaces (APIs)

## 2.1 - Downloading climate data from the Internet

We will now explore real world data, namely a dataset of climate information of 5 cities in Denmark between 1980-2018. The original raw-data was originally obtained from [National Climatic Data Center (NCDC)](https://www7.ncdc.noaa.gov/CDO/cdoselect.cmd).

In particular, the selected cities in Denmark are:
- Aalborg, 
- Aarhus, 
- Esbjerg, 
- Odense. 
- Roskilde

In the following, we will download the dataset using an API instead of loading a local source file.

### What is an API? 
Application Programming Interface (API) is a communication protocol between the user and the server (i.e., web server) that enables transmittion of data. The user making a request to an API server can download the desidered resources. 

### How to make a request in python?
There are many different way to request data. 
In the following, we will use the package **urllib** that collects several modules for working with URLs.

In particular, [urlib.request](https://docs.python.org/3/library/urllib.request.html#module-urllib.request) is a module for opening and reading URLs.


### - Download climate data


The following functions allow to download and store the dataset in a specific folder.

How to use:

- Set the source (i.e., URL) of the desidered dataset in `data_url`.
- Set the `data_dir` variable with the local directory where to store the data.
- Call `download_and_extract()` to download the dataset if it is not already located in the given data_dir.
- Load the data in the interactive Python notebook so ic can be used in your scripts (you can use `load_original_data()`).

In [ ]:
#@title Functions to download data 
########################################################################
#
# This file is a partially modified version of one of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2018 by Magnus Erik Hvass Pedersen
#
########################################################################

import sys
import os
import urllib.request
import tarfile
import zipfile



def _print_download_progress(count, block_size, total_size):
    """
    Function used for printing the download progress.
    Used as a call-back function in maybe_download_and_extract().
    """

    # Percentage completion.
    pct_complete = float(count * block_size) / total_size

    # Limit it because rounding errors may cause it to exceed 100%.
    pct_complete = min(1.0, pct_complete)

    # Status-message. Note the \r which means the line should overwrite itself.
    msg = "\r- Download progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()


def download(base_url, filename, download_dir):
    """
    Download the given file if it does not already exist in the download_dir.
    :param base_url: The internet URL without the filename.
    :param filename: The filename that will be added to the base_url.
    :param download_dir: Local directory for storing the file.
    """

    # Path for local file.
    save_path = os.path.join(download_dir, filename)

    # Check if the file already exists, otherwise we need to download it now.
    if not os.path.exists(save_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        print("Downloading", filename, "...")

        # Download the file from the internet.
        url = base_url + filename
        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=save_path,
                                                  reporthook=_print_download_progress)

        print(" Done!")


def download_and_extract(url, download_dir):
    """
    Download and extract the data if it doesn't already exist.
    Assumes the url is a tar-ball file.
    :param url:
        Internet URL for the tar-file to download.
    :param download_dir:
        Directory where the downloaded file is saved.
    """

    # Filename for saving the file downloaded from the internet.
    # Use the filename from the URL and add it to the download_dir.
    filename = url.split('/')[-1]
    file_path = os.path.join(download_dir, filename)

    # Check if the file already exists.
    # If it exists then we assume it has also been extracted,
    # otherwise we need to download and extract it now.
    if not os.path.exists(file_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        # Download the file from the internet.
        file_path, _ = urllib.request.urlretrieve(url=url,
                                                  filename=file_path,
                                                  reporthook=_print_download_progress)

        print()
        print("Download finished. Extracting files.")

        if file_path.endswith(".zip"):
            # Unpack the zip-file.
            zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
        elif file_path.endswith((".tar.gz", ".tgz")):
            # Unpack the tar-ball.
            tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

        print("Done.")
    else:
        print("Data has apparently already been downloaded and unpacked.")
        

First, we define where we want to download the dataset (you are free to choose another directory) and the URL of the dataset to be downloaded.

In [ ]:
# Location of the dataset on the internet.
data_url = "https://github.com/Hvass-Labs/weather-denmark/raw/master/weather-denmark.tar.gz"

# Local directory where you want to download and save the dataset.
data_dir = "weather-data/"

Now, we can download the dataset into the chosen local directory

In [ ]:
download_and_extract(url=data_url, download_dir=data_dir)

**[TO DO]:** Check the local folder? In which format the dataset has been downloaded?

**[TO DO]:** Select a format and load the dataset .

In [ ]:
### TODO


In [ ]:
### TODO

**More advanced solution for loading dataset**:

In this case, the dataset was downloaded in two different formats. 
We can define the path where the files have been stored as follows:
- **path_original_data_pickle( )** is the location of the original data in pickle format
- **path_original_data_csv( )** is the location the original data in .csv

In [ ]:
def path_original_data_pickle():
    return os.path.join(data_dir, "weather-denmark.pkl")

def path_original_data_csv():
    return os.path.join(data_dir, "weather-denmark.csv")

Now we can load the data in pickle format through the following function:

In [ ]:
def load_original_data():
    return pd.read_pickle(path_original_data_pickle())

In [ ]:
df = load_original_data()
df

### - Undestanding the data

**_[TO DO]_**: Have a quick overview of the downloaded dataset. 
In particular, focus on:

- Understanding the variables contained in the dataframe
- Check the DateTime for 2 cities

In [ ]:
### TODO

## 2.2 - Interacting with a server to query a subset of some data bank

We will now learn how to query a data bank to retrieve only the relevant proportion data for a task. We will be using the World Bank Data API to access World's renewable energy consumtpion data. 




You first need to install the World Bank API. In a terminal, run the following command:

``>> pip install wbdata``



In [ ]:
%pip install wbdata

In [ ]:
#@title Imports
import wbdata as wb
import datetime

### - Query the World Bank

* **_[TO DO]_**: Use the wbdata to retrieve renewable energy consumption data of all countries in 2015.

Hint: [wbdata documentation](https://wbdata.readthedocs.io/en/stable/)

In [ ]:
### TODO

### - Undestanding the data

* **_[TO DO]_**: Display different data points 

In [ ]:
### TODO

### - Exporting the data 

* **_[TO DO]_**: Save the retrieved renewable energy consumption data of all countries in 2015 in one .xlsx file
* **_[TO DO]_**: Save the renewable energy consumption data from 2006 to 2016 for each European country in individual .csv files (one per country)

In [ ]:
### TODO